In [1]:
import pandas as pd
import os
from datasets import Dataset, Audio
import speech_utils as su
from transformers import Wav2Vec2CTCTokenizer
from transformers import SeamlessM4TFeatureExtractor
from transformers import Wav2Vec2BertProcessor
from transformers import Wav2Vec2BertForCTC
from transformers import TrainingArguments
from transformers import Trainer
from data_collator import DataCollatorCTCWithPadding


processor_with_lm_path = '/home/kd/Desktop/proj/apr/speech_pa/wav2vec2-bert-pa-lm-processor-all_3'
model_path = '/home/kd/Desktop/proj/apr/speech_pa/wav2vec2-bert-pa_5/checkpoint-4000'
tokenizer_path = './'

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(tokenizer_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained(model_path)

for_lm = False
if processor_with_lm_path is not None:
    from m4t_processor_with_lm import M4TProcessorWithLM
    processor = M4TProcessorWithLM.from_pretrained(processor_with_lm_path)
    for_lm = True
else:
    processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

model = Wav2Vec2BertForCTC.from_pretrained(
    model_path, ignore_mismatched_sizes=True,
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    add_adapter=True,
    use_intermediate_ffn_before_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

/home/kd/Desktop/proj/apr/speech_pa/speech_utils.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/kd/anaconda3/envs/ai4bharat/lib/python3.12/site-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading the LM will be faster if you build a binary fil

In [ ]:
model.push_to_hub('kdcyberdude/w2v-bert-punjabi')
processor.push_to_hub('kdcyberdude/w2v-bert-punjabi')